# 우리가 다운 받은 이미지들의 이름을 우리가 원하는 형식으로 변경합니다

In [9]:
import os

# 이미지가 저장된 폴더 경로
images_dir = '/data/ephemeral/home/level2-cv-datacentric-cv-12/code/data/vietnamese2_receipt/img/train'  # 실제 이미지 폴더 경로로 변경하세요

# 파일 이름 변경 시작
for idx, filename in enumerate(sorted(os.listdir(images_dir)), start=1):
    # 이미지 파일만 대상으로 하기 위해 확장자 검사
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # 새 파일 이름 생성 (순서대로 번호 지정)
        new_filename = f"extractor.custom_vi.in_house.appen_{idx:06d}_page0001.jpg"  # .jpg 확장자에 맞게 설정

        # 현재 파일 경로와 새 파일 경로
        src = os.path.join(images_dir, filename)
        dst = os.path.join(images_dir, new_filename)

        # 파일 이름 변경
        os.rename(src, dst)
        print(f"{filename} -> {new_filename}")  # 변경 내역 출력

print("파일 이름 변경 완료")

09-03-19-receipt.jpg -> extractor.custom_vi.in_house.appen_000001_page0001.jpg
0AWSJD80F408FAA638B508lv.jpg -> extractor.custom_vi.in_house.appen_000002_page0001.jpg
0D3F76F745216DD05031B2sx.jpg -> extractor.custom_vi.in_house.appen_000003_page0001.jpg
1-Figure1-1.png -> extractor.custom_vi.in_house.appen_000004_page0001.jpg
29301400717_1df61876ca_z.jpg -> extractor.custom_vi.in_house.appen_000006_page0001.jpg
43372305725_ec95601bab_b.jpg -> extractor.custom_vi.in_house.appen_000007_page0001.jpg
43372307115_582077fff2_b.jpg -> extractor.custom_vi.in_house.appen_000008_page0001.jpg
75006-Thuy-Long.jpg -> extractor.custom_vi.in_house.appen_000009_page0001.jpg
IMG_3344-scaled.jpg -> extractor.custom_vi.in_house.appen_000010_page0001.jpg
das-konto.jpg -> extractor.custom_vi.in_house.appen_000012_page0001.jpg
fantastic-vietnamese.jpg -> extractor.custom_vi.in_house.appen_000013_page0001.jpg
images (1).jpeg -> extractor.custom_vi.in_house.appen_000014_page0001.jpg
images (2).jpeg -> extracto

# Easy OCR을 활용하여 임의의 사진에 Annotation을 적용하게 됩니다.
# 여기서 이미지가 너무 작거나 텍스트가 없는 경우 이를 제외하기 위한 코드 ( 박스가 없으면 train에 사용하지 않는다 )

In [10]:
import easyocr
import cv2
import numpy as np
import os
import json
from tqdm import tqdm

# EasyOCR 리더 초기화 (한국어 'ko' 사용, GPU 사용 가능 시 gpu=True로 설정)
reader = easyocr.Reader(['ko'], gpu=True)

def process_images_to_ufo_format(images_dir, output_json_path):
    """
    이미지 폴더 내 모든 이미지에 대해 텍스트 박스를 UFO 포맷으로 JSON에 저장합니다.
    
    Args:
        images_dir (str): 이미지가 저장된 폴더 경로.
        output_json_path (str): 결과 JSON 파일 저장 경로.
    """
    ufo_data = {"images": {}}

    # 이미지 폴더 내 모든 이미지 처리
    for filename in tqdm(os.listdir(images_dir), desc="Processing images"):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(images_dir, filename)
            image = cv2.imread(image_path)
            
            if image is None:
                print(f"이미지를 불러올 수 없습니다: {image_path}")
                continue

            # EasyOCR로 텍스트 검출
            results = reader.readtext(image)
            
            # 박스가 없으면 건너뛰기
            if not results:
                print(f"No text boxes found in image: {filename}")
                continue

            # 이미지 정보 초기화
            image_info = {
                "paragraphs": {},  # 빈 paragraphs 필드 추가
                "words": {}
            }

            for idx, (bbox, _, _) in enumerate(results):
                # UFO 포맷의 vertices 좌표를 맞추기 위한 변환
                vertices = [[float(coord[0]), float(coord[1])] for coord in bbox]
                
                # 결과를 이미지 내 인덱스별로 저장 (0001, 0002, ...)
                word_id = f"{idx:04d}"
                image_info["words"][word_id] = {
                    "transcription": "",  # 빈 transcription 필드 추가
                    "points": vertices
                }

            # 이미지별 텍스트 박스 정보 저장
            ufo_data["images"][filename] = image_info

    # JSON 파일로 저장
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(ufo_data, f, ensure_ascii=False, indent=4)

    print(f"UFO 포맷 JSON 파일이 '{output_json_path}'에 저장되었습니다.")

# 사용 예제
images_dir = '/data/ephemeral/home/level2-cv-datacentric-cv-12/code/data/vietnamese2_receipt/img/train'  # 이미지 폴더 경로
output_json_path = '/data/ephemeral/home/level2-cv-datacentric-cv-12/code/data/vietnamese2_receipt/ufo/train.json'  # 결과 JSON 파일 경로

# 이미지들을 UFO 포맷 JSON 파일로 저장
process_images_to_ufo_format(images_dir, output_json_path)

Processing images: 100%|██████████| 29/29 [00:40<00:00,  1.40s/it]

UFO 포맷 JSON 파일이 '/data/ephemeral/home/level2-cv-datacentric-cv-12/code/data/vietnamese2_receipt/ufo/train.json'에 저장되었습니다.
